In [210]:
import tensorflow as tf
from tensorflow import keras
import cv2
import numpy as np
import os
import matplotlib.pyplot as plt
from glob import glob

In [3]:
model = tf.keras.models.load_model("unet.h5")

In [201]:
img = cv2.imread(r"D:\paper implementation\Refined image segmentation for calorie estimation of multiple dish food item\images\dataset\Unet\test_images\2994639.jpg")

In [202]:
def img_processing(img):
    img = img / 255
    img = cv2.resize(img, (256,256))
    img = img.astype(np.float32)
    img = np.expand_dims(img, axis = 0)
    return img


def show(img):
    cv2.imshow("win", img)
    cv2.waitKey(0)

In [203]:
img_contours = img.copy()
img = img_processing(img)
mask = model.predict(img)

In [204]:
mask = mask[0]
mask = mask * 255
mask = cv2.resize(mask, (256,256))
mask.shape

(256, 256)

In [205]:
mask = np.uint8(mask)
edges = cv2.Canny(image=mask, threshold1=100, threshold2=200)
kernel = np.ones((4,4))
dilate = cv2.dilate(edges, kernel)
img_contours = cv2.resize(img_contours, (256,256))

In [206]:
contours,_ = cv2.findContours(dilate, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)
calories = 42
mass = 3700
for cnt in contours:
    area = cv2.contourArea(cnt)
#     cv2.drawContours(img_contours, cnt, -1, (0,255,0), 2)
    if(area > 1000):
        cal_estimation = (area * 42) // mass
        peri = cv2.arcLength(cnt, True)
        approx = cv2.approxPolyDP(cnt, 0.02*peri, True)
        x, y, w, h = cv2.boundingRect(approx)
        cv2.rectangle(img_contours, (x,y), (x+w, y+h), (0,255,0), 2)
        cv2.putText(img_contours, str(cal_estimation), (x+10,y+20), cv2.FONT_HERSHEY_DUPLEX, 0.5, (0,0,255), 2)
    

In [207]:
show(img_contours)

In [88]:
img_contours.shape

(256, 256, 3)

(256, 256)

In [211]:
test_images = glob("test_images/*")

In [229]:
test_images = glob("test_images/*")
k = 0
for path in test_images:
    total_cal = 0
    x = cv2.imread(path, cv2.IMREAD_COLOR)
    img_contours = x.copy()
    img = img_processing(x)
    
    mask = model.predict(img)
    
    mask = mask[0]
    mask = mask * 255
    mask = cv2.resize(mask, (256,256))
    mask = np.uint8(mask)
    edges = cv2.Canny(image=mask, threshold1=100, threshold2=200)
    kernel = np.ones((4,4))
    dilate = cv2.dilate(edges, kernel)
    img_contours = cv2.resize(img_contours, (256,256))
    
    contours,_ = cv2.findContours(dilate, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)
    calories = 42
    mass = 3700
    for cnt in contours:
        area = cv2.contourArea(cnt)
        if(area > 1000):
            cal_estimation = (area * 42) // mass
            total_cal+=cal_estimation
            peri = cv2.arcLength(cnt, True)
            approx = cv2.approxPolyDP(cnt, 0.02*peri, True)
            x, y, w, h = cv2.boundingRect(approx)
            cv2.rectangle(img_contours, (x,y), (x+w, y+h), (0,255,0), 2)
            cv2.putText(img_contours, str(cal_estimation), (x+10,y+20), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0,0,255), 1)
            
    cv2.putText(img_contours, str(total_cal), (5,30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0,0,255), 2)

    cv2.imwrite(f"mask/{k + 50}.png", img_contours)
    k+=1